In [2]:
from bs4 import BeautifulSoup
import re

# si ya guardaste el HTML con Selenium:
with open("Le-Male-Elixir-81642.html", "r", encoding="utf-8") as f:
    html = f.read()

soup = BeautifulSoup(html, "html.parser")   # o "html.parser" si no tienes lxml
body = soup.body

In [3]:
div_base = soup.select_one(
    "body > div#app > div.off-canvas-wrapper.grid-container > div#main-content > div.grid-x.grid-margin-x > div.small-12.medium-12.large-9.cell > div.grid-x.bg-white.grid-padding-x.grid-padding-y"
)
div_base

<div class="grid-x bg-white grid-padding-x grid-padding-y" itemscope="" itemtype="http://schema.org/Product" style="width: 100%; position: relative;"><div class="cell small-12" id="toptop" style="position: relative;"><h1 class="text-center medium-text-left" itemprop="name">Le Male Elixir Jean Paul Gaultier <small style="white-space: nowrap;">for men</small></h1><a download="Le-Male-Elixir-Jean-Paul-Gaultier-for-men-perfume-card.jpg" href="https://www.fragrantica.com/mdimg/perfume-social-cards/en-p_c_81642.jpeg" title="Le Male Elixir Jean Paul Gaultier for men Perfume Card" type="image/jpeg"><svg class="size-6" fill="none" stroke="currentColor" stroke-width="1.5" style="color: rgb(10, 10, 10); cursor: pointer; position: absolute; right: 11px; top: 46px; width: 1.6rem; height: 1.7rem;" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><svg class="size-6" fill="none" stroke="currentColor" stroke-width="1.5" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path d="m2.25 15.75 5

In [4]:
name = div_base.select_one(
    "div#toptop > h1.text-center.medium-text-left"
)

name_text = name.text.split(" for ")[0]
gender = name.text.split(" for ")[1]

print(name_text, gender)

Le Male Elixir Jean Paul Gaultier men


In [5]:
brand = div_base.select_one(
    "div.cell.small-12 > div.grid-x.grid-margin-x.grid-margin-y > div.cell.small-6.text-center > p > a > span"
)
brand.text

'Jean Paul Gaultier'

In [6]:
main_accords = div_base.select(
    "div.cell.small-12 > div.grid-x.grid-margin-x.grid-margin-y > div.cell.small-6.text-center > div.grid-x"
)

main_accords = main_accords[1]

main_accords = main_accords.select("div.cell.accord-box")

for accord in main_accords:
    print(accord.text)

vanilla
sweet
honey
aromatic
amber
lavender
tobacco
green
fresh spicy
powdery


In [7]:
ideal_times = {}

time = div_base.select(
    "div.cell.small-12"
)

time = time[1]

time = time.select(
    "div.grid-x.grid-margin-x.grid-margin-y"
)

time = time[3]

time = time.select(
    "div.cell.small-6"
)

time = time[1].select_one(
    "div"
)

time = time.select(
    "[index]"
)

for t in time:
    t_name = t.select_one(
        "div.show-for-medium"
    )

    t_name = t_name.select_one("span").text

    t_percentage = t.select_one(
        "div.voting-small-chart-size > div > div"
    )

    t_percentage = t_percentage.get("style","")
    m = re.search(r"width\s*:\s*([^;]+)", t_percentage, re.I)
    width = m.group(1).strip() if m else None

    ideal_times[t_name] = width

ideal_times

{'winter': '100%',
 'spring': '29.5621%',
 'summer': '13.78%',
 'fall': '78.7169%',
 'day': '37.8412%',
 'night': '83.1232%'}

In [8]:
notes_number = div_base.select(
    "div.cell.small-12 > div#pyramid.grid-x.grid-padding-y > div.cell > div > div"
)

elements = [str(tag.name) for tag in notes_number[1].find_all(True, recursive=True)]
indices_h4 = [indice for indice, elemento in enumerate(elements) if elemento == 'h4']

top_notes_number = elements[indices_h4[0]:indices_h4[1]].count("span")
middle_notes_number = elements[indices_h4[1]:indices_h4[2]].count("span")
base_notes_number = elements[indices_h4[2]:].count("span")


In [9]:
total_notes = []

notes = div_base.select(
    "div.cell.small-12 > div#pyramid.grid-x.grid-padding-y > div.cell > div > div > div > div > div > div"
)

for note in notes:
    if note.text != "":
        total_notes.append(note.text)

notes_final = {
    "top_notes": total_notes[:top_notes_number],
    "middle_notes": total_notes[top_notes_number:top_notes_number + middle_notes_number],
    "base_notes": total_notes[-base_notes_number:]
}

notes_final

{'top_notes': ['Lavender', 'Mint'],
 'middle_notes': ['Vanilla', 'Benzoin'],
 'base_notes': ['Honey', 'Tonka Bean', 'Tobacco']}

In [10]:
total_notes[top_notes_number:top_notes_number + middle_notes_number]

['Vanilla', 'Benzoin']

In [11]:
total_notes[-base_notes_number:]

['Honey', 'Tonka Bean', 'Tobacco']

In [12]:
# pip install requests
import os, mimetypes, requests

def download_image(url, out_dir="images", filename=None,
                   referer="https://www.fragrantica.com/perfume/Jean-Paul-Gaultier/Le-Male-Elixir-81642.html"):
    os.makedirs(out_dir, exist_ok=True)
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": referer,  # importante para algunos CDNs
    }
    with requests.get(url, headers=headers, stream=True, timeout=20) as r:
        r.raise_for_status()
        # nombre por defecto desde la URL
        name = filename or url.split("/")[-1].split("?")[0]
        # asegura extensión correcta desde el Content-Type
        ctype = (r.headers.get("Content-Type") or "").split(";")[0]
        ext = mimetypes.guess_extension(ctype) or ".jpg"
        if "." not in os.path.basename(name):
            name += ext
        path = os.path.join(out_dir, name)
        with open(path, "wb") as f:
            for chunk in r.iter_content(8192):
                if chunk:
                    f.write(chunk)
    return path

# Ejemplo con tu JSON
data = {
    "img_url": "https://fimgs.net/mdimg/perfume-thumbs/375x500.81642.jpg"
}
saved_path = download_image(data["img_url"])
print("Guardada en:", saved_path)


Guardada en: images/375x500.81642.jpg


/Users/juanaleman/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [15]:
url_master = "https://www.fragrantica.com/perfume/Zimaya/Mazaaj-Infused-97103.html"
out = url_master.split("/")[-1]
image_filename = url_master.split("/")[-1][:5]

print(out)
print(image_filename)

Mazaaj-Infused-97103.html
Mazaa
